<a href="https://colab.research.google.com/github/123prashanth123/Fault-Detection-System/blob/Colabs/Feature%20Vector%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Setup

In [ ]:
from IPython.display import clear_output
from google.colab import drive

drive.mount("/content/gdrive")

clear_output()

In [ ]:
import os
import re
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random as r

import torch
from torch import nn
from torchvision import models, transforms

from sklearn.metrics.pairwise import manhattan_distances

import warnings
warnings.filterwarnings("ignore")

In [ ]:
PATH = "/content/gdrive/My Drive/Datasets/FDS"
POSITIVE_DIR = os.path.join(PATH, "Positive Images")
NEGATIVE_DIR = os.path.join(PATH, "Negative Images")
SEED = 0

def breaker(num=50, char='*'):
    print("\n" + num*char + "\n")

def preprocess(image):
    image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2RGB)
    image = cv2.resize(src=image, dsize=(256, 256), interpolation=cv2.INTER_AREA)
    h, w, _ = image.shape
    relief = 112
    cx, cy = w // 2, h // 2
    return image[cy - relief:cy + relief, cx - relief:cx + relief, :]

def morph(image=None):
    return cv2.morphologyEx(src=image, op=cv2.MORPH_CLOSE, kernel=np.ones((15, 15)))

# Feature Extraction Model

In [ ]:
def build_model():
    breaker()
    print("Building Feature Extractor ....")

    class FeatureExtractor(nn.Module):
        def __init__(self, name="resnet50"):
            super(FeatureExtractor, self).__init__()

            self.name = name
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            self.transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])

            if re.match(r"resnet50", name, re.IGNORECASE):
                self.model = models.resnet50(pretrained=True, progress=True)
                self.model = nn.Sequential(*[*self.model.children()][:-1])
                self.model.add_module("Flatten", nn.Flatten())
            
            elif re.match(r"vgg16bn", name, re.IGNORECASE):
                self.model = models.vgg16_bn(pretrained=True,progress=True)
                self.model = nn.Sequential(*[*self.model.children()][:-2])
                self.model.add_module("Adaptive Average Pool", nn.AdaptiveAvgPool2d(output_size=(2, 2)))
                self.model.add_module("Flatten", nn.Flatten())
        

        def forward(self, x):
            return self.model(x)
        

        def normalize(self, x):
            for i in range(x.shape[0]):
                x[i] = (x[i] - torch.min(x[i])) / (torch.max(x[i]) - torch.min(x[i]))
            return x
        

        def get_features(self, image):
            self.to(self.device)
            with torch.no_grad():
                features = self(self.transform(image).to(self.device).unsqueeze(dim=0))
            return self.normalize(features).detach().cpu(), self.normalize(features).detach().cpu().numpy()


    model = FeatureExtractor(name="vgg16bn")
    model.eval()

    return model

# Analyze Features

In [ ]:
def analyze_features(model=None, pos_path=None, neg_path=None):
    breaker()
    print("Analyzing Features ....")
    breaker()

    def get_features(model, path):
        names = os.listdir(path)

        torch_features = []
        numpy_features = []
        for name in names:
            image = preprocess(cv2.imread(os.path.join(path, name), cv2.IMREAD_COLOR))
            t, n = model.get_features(image)
            torch_features.append(t)
            numpy_features.append(n)
        return torch_features, numpy_features
        
    p_features, _ = get_features(model, pos_path)
    n_features, _ = get_features(model, neg_path)

    countp = 1
    for p_feat in p_features:
        countn = 1
        for n_feat in n_features:
            cos_sim = nn.CosineSimilarity()(p_feat, n_feat).item()
            euc_dis = nn.PairwiseDistance()(p_feat, n_feat).item()
            print("Positive Image {}, Negative Image {}, Cosine Similarity : {:.5f}, Euclidean Distance : {:.5f}".format(countp, countn, cos_sim, euc_dis))
            countn += 1
        countp += 1
        breaker()

# Main

In [ ]:
def main():
    model = build_model()
    analyze_features(model, POSITIVE_DIR, NEGATIVE_DIR)
main()


**************************************************

Building Feature Extractor ....


Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth




**************************************************

Analyzing Features ....

**************************************************

Positive Image 1, Negative Image 1, Cosine Similarity : 0.71755, Euclidean Distance : 4.81238
Positive Image 1, Negative Image 2, Cosine Similarity : 0.52099, Euclidean Distance : 5.77033
Positive Image 1, Negative Image 3, Cosine Similarity : 0.50352, Euclidean Distance : 6.21682
Positive Image 1, Negative Image 4, Cosine Similarity : 0.49885, Euclidean Distance : 5.44535
Positive Image 1, Negative Image 5, Cosine Similarity : 0.72038, Euclidean Distance : 4.29459
Positive Image 1, Negative Image 6, Cosine Similarity : 0.52167, Euclidean Distance : 5.48309
Positive Image 1, Negative Image 7, Cosine Similarity : 0.57170, Euclidean Distance : 5.28908
Positive Image 1, Negative Image 8, Cosine Similarity : 0.57174, Euclidean Distance : 5.84452
Positive Image 1, Negative Image 9, Cosine Similarity : 0.51478, Euclidean Distance : 5.07255
Positive Image 1, Negat